In [1]:
import numpy as np
from scipy.special import hermite
from math import factorial, sqrt, pi

def phi(n, x):
    Hn = hermite(n)
    norm = 1.0 / sqrt((2**n) * factorial(n) * sqrt(pi))
    return norm * Hn(x) * np.exp(-x**2 / 2)


In [2]:
def K(x, y, N):
    return sum(phi(n, x) * phi(n, y) for n in range(N))


In [3]:
# Discretize space
xgrid = np.linspace(-4, 4, 200)
dx = xgrid[1] - xgrid[0]

# Build kernel matrix
N = 5
Kmat = np.zeros((len(xgrid), len(xgrid)))

for i, x in enumerate(xgrid):
    for j, y in enumerate(xgrid):
        Kmat[i, j] = K(x, y, N) * dx


In [4]:
eigvals, eigvecs = np.linalg.eigh(Kmat)
print(np.round(eigvals, 6))


[-0.       -0.       -0.       -0.       -0.       -0.       -0.
 -0.       -0.       -0.       -0.       -0.       -0.       -0.
 -0.       -0.       -0.       -0.       -0.       -0.       -0.
 -0.       -0.       -0.       -0.       -0.       -0.       -0.
 -0.       -0.       -0.       -0.       -0.       -0.       -0.
 -0.       -0.       -0.       -0.       -0.       -0.       -0.
 -0.       -0.       -0.       -0.       -0.       -0.       -0.
 -0.       -0.       -0.       -0.       -0.       -0.       -0.
 -0.       -0.       -0.       -0.       -0.       -0.       -0.
 -0.       -0.       -0.       -0.       -0.       -0.       -0.
 -0.       -0.       -0.       -0.       -0.       -0.       -0.
 -0.       -0.       -0.       -0.       -0.       -0.       -0.
 -0.       -0.       -0.       -0.       -0.       -0.       -0.
 -0.       -0.       -0.       -0.       -0.       -0.       -0.
 -0.       -0.        0.        0.        0.        0.        0.
  0.        0.        0. 

In [5]:
import mpmath as mp

def phi(n, x):
    """
    Harmonic oscillator eigenfunction Φ_n(x)
    """
    Hn = mp.hermite(n, x)
    norm = mp.sqrt(1 / (mp.sqrt(mp.pi) * (2**n) * mp.factorial(n)))
    return norm * Hn * mp.e**(-x**2 / 2)


In [6]:
def kernel(x , y, N):
    """
    Kernel K(x, y) = Σ_{n=0}^{N-1} Φ_n(x) Φ_n(y)
    """
    return mp.nsum(phi(n, x) * phi(n, y) for n in range(N))

In [7]:
def kernel_matrix(xgrid, N):
    dx = xgrid[1] - xgrid[0]
    M = mp.zeros(len(xgrid), len(xgrid))

    for i, x in enumerate(xgrid):
        for j, y in enumerate(xgrid):
            M[i, j] = kernel(x, y, N) * dx

    return M


In [8]:
mp.mp.dps = 100

# Parameters
N = 5
xgrid = [mp.mpf(x) for x in mp.linspace(-4, 4, 120)]

# Build kernel matrix
Kmat = kernel_matrix(xgrid, N)

# Eigenvalues of integral operator
eigvals, eigvecs = mp.eig(Kmat)

# Print eigenvalues
for ev in eigvals:
    print(mp.nstr(ev, 20))


TypeError: cannot unpack non-iterable NoneType object

In [9]:
def overlap_matrix(N, a, b, Nprec=200):
    mp.mp.dps = Nprec

    A = mp.zeros(N, N)

    for m in range(N):
        for n in range(N):
            integrand = lambda x: phi(m, x) * phi(n, x)
            A[m, n] = mp.quad(integrand, [a, b])

    return A


In [10]:
# Example
mp.mp.dps = 200
N = 6
a, b = -1, 1

A = overlap_matrix(N, a, b)

eigvals, eigvecs = mp.eig(A)

for ev in eigvals:
    print(mp.nstr(ev, 20))


0.98683424421728684409
0.27769727195670347137
0.00095222468751206989765
0.8516310362987759655
0.000054359986471891710279
0.050579284333234162727


In [11]:
mp.mp.dps = 200

# A = overlap_matrix(...)   # computed earlier

eigvals_A, _ = mp.eig(A)

tol = mp.mpf('1e-40')
zeta = [ev for ev in eigvals_A if tol < ev < 1 - tol]

epsilon = [mp.log((1 - ev) / ev) for ev in zeta]

print("Entanglement Hamiltonian eigenvalues:")
for e in epsilon:
    print(mp.nstr(e, 25))


Entanglement Hamiltonian eigenvalues:
-4.316882886554486491413523
0.9559127713756500191219414
6.955756856384732059126805
-1.74745120766519226935094
9.819827856078780171104364
2.932309933687905178069169
